In [1]:
import math
import numpy as np
import wandb
import pickle
import os
import shapely.wkt as wkt
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from torch_geometric.transforms import LineGraph

import gzip
import xml.etree.ElementTree as ET

import torch
import torch_geometric
from torch_geometric.data import Data

import processing_io as pio

from gnn_architectures import MyGnn

import gnn_io as gio
import gnn_architectures as garch

In [2]:
test_data = "../data/test_data/pop_1pm_policy_in_zone_1.geojson"
base_case = "../data/test_data/gdf_basecase_mean_pop_1pm.geojson"
test_data = gpd.read_file(test_data)
base_case = gpd.read_file(base_case)

In [3]:
test_data_object = pio.create_test_data_object(base_case= base_case, test_data = test_data)

In [4]:
test_data_list = [test_data_object]

## 2. Load data and model

In [14]:
dataset_only_relevant_dimensions = gio.cut_dimensions(dataset=test_data_list, indices_of_dimensions_to_keep=[0, 1])
dataset_normalized = gio.normalize_dataset(dataset_only_relevant_dimensions)

In [15]:
# Apply the function to the dataset
dataset_normalized[0]

Data(edge_index=[2, 59135], pos=[31216, 2], num_nodes=31140, x=[31216, 2], y=[31216, 1])

In [16]:
test_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=16, train_ratio=0, is_test=True)

Total dataset length: 1


In [17]:
wandb.login()
wandb.init(project="test_with_1pct_data")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.load('trained_models/model_with_feature_base_flow.pth')
model.eval()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


MyGnn(
  (pointLayer): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
  ), global_nn=Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
  ))
  (output_layer): GATConv(16, 1, heads=1)
)

## 4. Test the model

In [24]:
# Function to evaluate the model
def evaluate(model, test_dl, device):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():  # Disable gradient computation
        for data in test_dl:
            input_node_features, targets = data.x.to(device), data.y.to(device)
            print(targets.shape)
            outputs = model(data)
            print(outputs.shape)
            loss = torch.nn.MSELoss()(outputs, targets)
            print(loss.item())
            test_loss += loss.item()
    avg_test_loss = test_loss / len(test_dl)
    
    # Log the test loss to Wandb
    wandb.log({"test_loss": avg_test_loss})
    return avg_test_loss, outputs

# Evaluate the loaded model
test_loss, outputs = evaluate(model, test_dl, device)
print(f'Test Loss: {test_loss}')

torch.Size([31216, 1])
torch.Size([31216, 1])
0.0020746015943586826
Test Loss: 0.0020746015943586826


In [26]:
import joblib

def recreate_original_y_values(dataset):
    # Load the saved scaler
    scaler = joblib.load('y_scaler.pkl')

    # Apply the inverse transform to each data instance
    for data in dataset:
        data.y = torch.tensor(scaler.inverse_transform(data.y.numpy()), dtype=torch.float)

    return dataset

torch.Size([31216, 1])

In [27]:
test_data_object

Data(edge_index=[2, 59135], pos=[31216, 2], num_nodes=31140, x=[31216, 2], y=[31216, 1])

In [42]:
# Assuming 'highway_mapping' and 'encode_modes' are defined as in your context
highway_mapping = {
    'residential': 0, 'tertiary': 1, 'living_street': 2, 'secondary': 3, 
    'primary': 4, 'trunk_link': 5, 'primary_link': 6, 'motorway': 7, 
    'service': 8, 'unclassified': 9, 'secondary_link': 10, 
    'pedestrian': 11, 'trunk': 12, 'motorway_link': 13, 
    'construction': 14, 'tertiary_link': 15, np.nan: -1
}

def data_to_geodataframe(data, original_gdf):
    # Extract the edge index and node features
    node_features = data.x.numpy()
    target_values = data.y.numpy()

    # Create edge data
    edge_data = {
        'from_node': original_gdf["from_node"].values,
        'to_node': original_gdf["to_node"].values,
        'capacity': node_features[:, 0],  # Assuming capacity is the first feature
        'vol_base_case': node_features[:, 1],  # Assuming vol_base_case is the second feature
        'vol_car': target_values.squeeze()  # Assuming target values are car volumes
    }
    # Convert to DataFrame
    edge_df = pd.DataFrame(edge_data)
    
    # Create LineString geometry
    edge_df['geometry'] = original_gdf["geometry"].values

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(edge_df, geometry='geometry')

    return gdf

# Example usage
# Assuming 'data' is your Data object and 'original_gdf' is the original GeoDataFrame
predicted_gdf = data_to_geodataframe(test_data_object, test_data)

(31216,)
(31216,)
(59135,)


In [43]:
predicted_gdf.head(10)

,from_node,to_node,capacity,vol_base_case,vol_car,from_coords,to_coords,geometry
0,24972409,24972408,0.006037,0.038307,0.037257,NaN,NaN,"LINESTRING (2.33869 48.85181, 2.33847 48.85181)"
1,5904976363,24983651,0.006037,0.049539,0.042657,NaN,NaN,"LINESTRING (2.33874 48.85242, 2.33872 48.85229)"
2,24983651,5904976363,0.018110,0.010830,0.006479,NaN,NaN,"LINESTRING (2.33872 48.85229, 2.33874 48.85242)"
3,664205947,24972376,0.018110,0.038909,0.032937,NaN,NaN,"LINESTRING (2.33994 48.85200, 2.33986 48.85181)"
4,24972376,24972375,0.006037,0.043923,0.038337,NaN,NaN,"LINESTRING (2.33986 48.85181, 2.33909 48.85152)"
5,324579210,4964831516,0.006037,0.029483,0.024298,NaN,NaN,"LINESTRING (2.34264 48.85031, 2.34258 48.85031)"
6,4964831516,24972333,0.006037,0.029483,0.024298,NaN,NaN,"LINESTRING (2.34258 48.85031, 2.33999 48.84994)"
7,24972382,4964831514,0.000000,0.000000,0.000000,NaN,NaN,"LINESTRING (2.34151 48.85112, 2.34273 48.85065)"
8,24972408,24972143,0.006037,0.092659,0.085313,NaN,NaN,"LINESTRING (2.33847 48.85181, 2.33840 48.85172)"
9,24972144,24972325,0.006037,0.036302,0.034017,NaN,NaN,"LINESTRING (2.33806 48.85065, 2.33842 48.84991)"
